 1. ENLACE DRIVE Y CARGA LIBRERÍAS:

In [19]:
import pandas as pd
import numpy as np
import gzip
import json
import spacy
import matplotlib.pyplot as plt
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re
import string
import logging
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from collections import Counter
from collections import defaultdict
from wordcloud import WordCloud
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.phrases import Phrases, Phraser
from time import time

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
!ls '/content/drive/MyDrive/Colab Notebooks'

Mounted at /content/drive/
df_clean.csv  reviews_Clothing.json.gz


2. CARGA DE ARCHIVO Y PUESTA ÓPTIMA

In [3]:
df_clean = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/df_clean.csv')

In [4]:
df_clean.shape

(278569, 6)

In [5]:
df_clean.head()

,Unnamed: 0,overall,reviewText,sentiment,assessment,tokenized_review
0,0,5.0,This is a great tutu and at a really great pri...,1,Positiva,"['This', 'is', 'a', 'great', 'tutu', 'and', 'a..."
1,1,5.0,I bought this for my 4 yr old daughter for dan...,1,Positiva,"['I', 'bought', 'this', 'for', 'my', '4', 'yr'..."
2,2,5.0,What can I say... my daughters have it in oran...,1,Positiva,"['What', 'can', 'I', 'say...', 'my', 'daughter..."
3,3,5.0,"We bought several tutus at once, and they are ...",1,Positiva,"['We', 'bought', 'several', 'tutus', 'at', 'on..."
4,4,5.0,Thank you Halo Heaven great product for Little...,1,Positiva,"['Thank', 'you', 'Halo', 'Heaven', 'great', 'p..."


In [6]:
df_clean = df_clean[['overall','reviewText','sentiment']]

In [7]:
df_clean.head()

,overall,reviewText,sentiment
0,5.0,This is a great tutu and at a really great pri...,1
1,5.0,I bought this for my 4 yr old daughter for dan...,1
2,5.0,What can I say... my daughters have it in oran...,1
3,5.0,"We bought several tutus at once, and they are ...",1
4,5.0,Thank you Halo Heaven great product for Little...,1


3. PREPROCESADO

Para realizar el preprocesado, valoramos varios factores a tener en cuenta:

* Codificación ASCII para eliminar caracteres extraños y signos de puntuación.
* Eliminación stop words, Tokenización, Lematización.
* Paso a minúsculas.
* Conversión números a texto.
* Eliminación token < 3 caracteres.

In [8]:
!pip install -U spacy download en_core_web_sm

In [9]:
import en_core_web_sm
import unicodedata
nlp = en_core_web_sm.load()
from bs4 import BeautifulSoup

In [10]:
def unicode_normalize(text):
    if not isinstance(text, str):  # Verifica si el texto es una cadena
        return ""  # Devuelve una cadena vacía en caso de NaN o números
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')


In [20]:
import en_core_web_sm
nlp = en_core_web_sm.load()

def normalizacion_textos(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar signos de puntuación y caracteres especiales
    text = re.sub(f"[{string.punctuation}]", " ", text)
    # Eliminar números
    text = re.sub(r"\d+", "", text)
    # Procesar el texto con spaCy
    doc = nlp(text)
    # Lematización y eliminación de stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.text.strip() != ""]
    # Unir tokens en un solo string limpio
    return " ".join(tokens)

# Aplicar preprocesamiento avanzado a todas las reviews
df_clean["cleaned_review"] = df_clean["reviewText"].dropna().apply(normalizacion_textos)

In [21]:
df_clean.head()

,overall,reviewText,sentiment,cleaned_review
0,5.0,This is a great tutu and at a really great pri...,1,great tutu great price doesn t look cheap m gl...
1,5.0,I bought this for my 4 yr old daughter for dan...,1,buy yr old daughter dance class wear today tim...
2,5.0,What can I say... my daughters have it in oran...,1,daughter orange black white pink think buy fuc...
3,5.0,"We bought several tutus at once, and they are ...",1,buy tutus get high review sturdy seemingly gir...
4,5.0,Thank you Halo Heaven great product for Little...,1,thank halo heaven great product little girl gr...


In [22]:
# Ahora la columna sentiment:
df_clean['sentiment'] = df_clean['overall'].apply(lambda x: 1 if x >= 4 else 0)
df_clean.head()

,overall,reviewText,sentiment,cleaned_review
0,5.0,This is a great tutu and at a really great pri...,1,great tutu great price doesn t look cheap m gl...
1,5.0,I bought this for my 4 yr old daughter for dan...,1,buy yr old daughter dance class wear today tim...
2,5.0,What can I say... my daughters have it in oran...,1,daughter orange black white pink think buy fuc...
3,5.0,"We bought several tutus at once, and they are ...",1,buy tutus get high review sturdy seemingly gir...
4,5.0,Thank you Halo Heaven great product for Little...,1,thank halo heaven great product little girl gr...


In [23]:
df_clean['sentiment'].value_counts()

,count
sentiment,
1,221495
0,57074


In [24]:
df_clean[df_clean['reviewText'].isna()].head()

,overall,reviewText,sentiment,cleaned_review


Guardo el data procesado para continuar en el siguiente notebook

In [27]:
df_clean.to_csv('/content/drive/MyDrive/Colab Notebooks/data_preprocessed.csv')